In [89]:
%matplotlib inline
import data
import builder
import pandas as pd

df = pd.DataFrame(data.build_dataset(10000))
df.head()

,holder,holder_index,amount,dispute_risk,incidents,age,holder_risk
0,PLATINUM,3,130.05,2,2,66,0
1,PLATINUM,3,192.87,2,0,41,0
2,PLATINUM,3,174.57,2,0,21,0
3,STANDARD,0,67.64,2,0,43,1
4,GOLD,2,195.93,3,3,54,2


## Dispute Risk (DR) model
Split the datasets:

In [90]:
from sklearn.model_selection import train_test_split

DR_outputs = df['dispute_risk']
DR_inputs = df[['amount', 'holder_index']]

DR_X_train, DR_X_test, DR_y_train, DR_y_test = train_test_split(DR_inputs, DR_outputs, test_size=0.4, random_state=23)

In [91]:
DR_lr_pipeline = builder.build_LR_pipeline(DR_X_train, DR_y_train)

In [92]:
DR_lr_predictions = DR_lr_pipeline.predict(DR_X_test)
print(DR_lr_pipeline.score(DR_X_test, DR_y_test))


0.8018183683186544


In [93]:
DR_rf_pipeline = builder.build_RF_pipeline(DR_X_train, DR_y_train)

/Users/rui/Sync/code/python/risk-pmml-builder/venv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [94]:
DR_rf_predictions = DR_rf_pipeline.predict(DR_X_test)
print(DR_rf_pipeline.score(DR_X_test, DR_y_test))

0.9999312191483045


In [95]:
DR_result = pd.DataFrame({'LR': DR_lr_predictions.astype(int), 'RF': DR_rf_predictions.astype(int), 'truth': DR_y_test})
DR_result.head(20)

,LR,RF,truth
6326,3,3,3
5185,2,1,1
2495,3,3,3
2765,2,2,2
9303,2,2,2
322,2,3,3
4164,2,3,3
7198,2,3,3
2626,1,2,2
4492,1,1,1


## Card Holder (CH) model

In [96]:
CH_outputs = df['holder_risk']
CH_inputs = df[['age', 'holder_index', 'incidents']]

CH_X_train, CH_X_test, CH_y_train, CH_y_test = train_test_split(CH_inputs, CH_outputs, test_size=0.4, random_state=23)

In [97]:
CH_lr_pipeline = builder.build_LR_pipeline(CH_X_train, CH_y_train)

In [98]:
CH_lr_predictions = CH_lr_pipeline.predict(CH_X_test)
print(CH_lr_pipeline.score(CH_X_test, CH_y_test))


0.6141092845378208


In [99]:
CH_rf_pipeline = builder.build_RF_pipeline(CH_X_train, CH_y_train)

/Users/rui/Sync/code/python/risk-pmml-builder/venv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [100]:
CH_rf_predictions = CH_rf_pipeline.predict(CH_X_test)
print(CH_rf_pipeline.score(CH_X_test, CH_y_test))

1.0


In [101]:
CH_result = pd.DataFrame({'LR': CH_lr_predictions.astype(int), 'RF': CH_rf_predictions.astype(int), 'truth': CH_y_test})
CH_result.head(20)

,LR,RF,truth
6326,1,3,3
5185,1,0,0
2495,1,0,0
2765,1,3,3
9303,1,3,3
322,0,0,0
4164,1,2,2
7198,0,1,1
2626,0,0,0
4492,0,0,0
